# Deployment 

https://github.com/topspinj/recommender-tutorial/blob/master/part-3-implicit-feedback-recommender.ipynb


We only use ALS (efficient)

Input: User Id --> Top 5 activity; (Apply real-time (new data), handle missing users, invalid input)

# Training, get the model (fast) 

## Settings

In [1]:
OUTPUT_CLEANED_FOLDER = "output/"

## Read data

In [2]:
import warnings
import os 
import pandas as pd 
from pandas.errors import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
warnings.simplefilter(action="ignore", category=DeprecationWarning)



df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))
df_user = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_info_deep_cleaned.csv"))
df_activity = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "activity_info_deep_cleaned.csv"))

/tmp/ipykernel_3304683/1710496557.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df_interaction = pd.read_csv(os.path.join(OUTPUT_CLEANED_FOLDER, "user_activity_interaction_deep_cleaned.csv"))


In [3]:
df_activity["id"]  = df_activity["id"].astype(str)
df_interaction = df_interaction.astype(str)
df_interaction["inter_score"] = 1
df_all = df_interaction.merge(df_activity, left_on=["activity_id"], right_on=["id"])[["user_id", "activity_id", "inter_score", "createdTime"]]
df_all["createdTime"] = pd.to_datetime(df_all["createdTime"])

In [4]:
df_all = df_all.drop_duplicates()

## Preprocessing

In [5]:
user_id_map = {i:v for i, v in enumerate(df_all["user_id"].unique())}
inverse_user_id_map = {v:i for i, v in enumerate(df_all["user_id"].unique())}
activity_id_map = {i:v for i, v in enumerate(df_all["activity_id"].unique())}
inverse_activity_id_map = {v: i for i, v in enumerate(df_all["activity_id"].unique())}

In [6]:
df_all_map = df_all.copy()

df_all_map['user_id'] = df_all_map['user_id'].map(inverse_user_id_map)
df_all_map['activity_id'] = df_all_map['activity_id'].map(inverse_activity_id_map)

In [7]:
print('df_all_map shape: ',df_all_map.shape)
print('number of users: ', len(user_id_map))
print('number of items: ', len(activity_id_map))
print('The sparse matrix is one with shape (%d , %d), with %d non-zero read_times'
      %(len(user_id_map),len(activity_id_map), df_all_map.shape[0]))

df_all_map shape:  (3286982, 4)
number of users:  59272
number of items:  4505
The sparse matrix is one with shape (59272 , 4505), with 3286982 non-zero read_times


## Training Model

In [8]:
import numpy as np

nb_users = len(user_id_map)
nb_articles = len(activity_id_map)
uim = np.zeros((nb_users, nb_articles), dtype=np.float32)
uim[df_all_map["user_id"], df_all_map["activity_id"]] = df_all_map["inter_score"]

In [9]:
uim.shape

(59272, 4505)

In [11]:
from scipy import sparse as sp
from implicit.als import AlternatingLeastSquares

uim = sp.csr_matrix(uim)
print('uim shape :', uim.shape)

model = AlternatingLeastSquares(factors=16, 
                                random_state=42,
                                num_threads=16,
                                alpha=128)

model.fit(user_items=uim, show_progress=True)

/home/ctv.sangdv/.conda/envs/recommender-d/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


/home/ctv.sangdv/.local/lib/python3.9/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 64 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


uim shape : (59272, 4505)


100%|██████████| 15/15 [00:08<00:00,  1.76it/s]


In [12]:
model.user_factors.shape[0], model.item_factors.shape[0]

(59272, 4505)

# Deployment Interface

## Settings

In [13]:
# Parameters
considered_most_recent_top = 20 # not include already engaged activities of a user 

recommended_topK = 5
recent_topK = 10
similar_userN = 10
# Input users --> number of enrolled activities, recent activities, recommendation, similar users

# Interface

In [14]:
def valid_get_user_id(user_id):
    import re 
    exception = Exception("Please provide valid USER ID.")
    
    user_id = str(user_id)
    pattern = r"^(201[2-9]|202[0-3])\d{4}P?$"

    if (not isinstance(user_id, int) and not isinstance(user_id, str)) or not bool(re.match(pattern, user_id)):
        raise(exception)

    if user_id not in inverse_user_id_map:
        raise(Exception(f"User ID: {user_id} not in database ..., you could choose random activities"))
    
    return user_id

In [15]:

def get_basic_info(user_id):
    URL_FORMAT = "https://ctsv.hust.edu.vn/#/hoat-dong/{}/chi-tiet"

    user_id = valid_get_user_id(user_id)
    user_id_mapped = inverse_user_id_map[user_id]
    user_id_interact_df = df_all_map[df_all_map["user_id"] == user_id_mapped].reset_index(drop=True)
    num_interactions  = len(user_id_interact_df)
    print(f"Student ID: {user_id}. Number of Engaged Events: {num_interactions}")

    cared_columns = ["id", "title", "type", "btc"]
    df_user_activity_merged_sort = df_activity.merge(user_id_interact_df["activity_id"].map(activity_id_map), 
                                                     left_on="id", 
                                                     right_on="activity_id").sort_values(by="createdTime", ascending=False)[cared_columns]
    
    recent_top_k_df = df_user_activity_merged_sort[:recent_topK]
    recent_top_k_df["url"] = recent_top_k_df["id"].apply(lambda x: URL_FORMAT.format(x))

    print(f"\n-- Recent Engaged Activities: ")
    for index, row in recent_top_k_df.reset_index().iterrows():
        print(f"{index+1}. {row['title']}\nBTC: {row['btc']}\nThể loại: {row['type']}\nLink: {row['url']}")


In [16]:
def get_recommend_activities(user_id):
    user_id = valid_get_user_id(user_id)
    user_id_mapped = inverse_user_id_map[user_id]

    URL_FORMAT = "https://ctsv.hust.edu.vn/#/hoat-dong/{}/chi-tiet"
    activity_list = df_all_map[["activity_id", "createdTime"]].drop_duplicates().sort_values(by="createdTime", ascending=False)["activity_id"].to_list()
    user_id_interact_df = df_all_map[df_all_map["user_id"] == user_id_mapped].reset_index(drop=True)
    count = 0 
    user_interact_set = set(user_id_interact_df["activity_id"].tolist())
    candidates_act_id = []

    for act in activity_list:
        if act not in user_interact_set:
            candidates_act_id.append(act)
            count += 1
            if count == considered_most_recent_top:
                break
            
    predictions = model.rank_items(user_id_mapped, uim, candidates_act_id)
    recommendation_list = predictions[0][:recommended_topK]
    recommendation_conf = predictions[1][:recommended_topK]
    recommendation_data = list(zip(recommendation_list, recommendation_conf))

    df_recommendation = pd.DataFrame(recommendation_data, columns=["activity_id", "confidence"])
    df_recommendation["activity_id"]  = df_recommendation["activity_id"].map(activity_id_map)

    cared_columns = ["id", "confidence", "title", "type", "btc"]
    df_user_activity_recommendation= df_activity.merge(df_recommendation, left_on="id", right_on="activity_id").sort_values(by="confidence", ascending=False)[cared_columns]
    recommend_top_k_df  = df_user_activity_recommendation[:recent_topK]
    recommend_top_k_df["url"]  = recommend_top_k_df["id"].apply(lambda x: URL_FORMAT.format(x))

    print(f"User {user_id}: Top {recommended_topK} recommender ...")
    for index, row in recommend_top_k_df.reset_index().iterrows():
        print(f"{index+1}.Title: {row['title']}\nBTC: {row['btc']}\nType: {row['type']}\nLink: {row['url']}\nConfidence: {row['confidence']:.4f}")
        print()

In [17]:
def get_similar_users(user_id):
    user_id = valid_get_user_id(user_id)
    user_id_mapped = inverse_user_id_map[user_id]

    # Similar users
    similar_user_N = model.similar_users(user_id_mapped, N=similar_userN, filter_users=user_id_mapped)

    similar_user_N_df = pd.DataFrame(list(zip(similar_user_N[0], similar_user_N[1])), columns=["id", "confidence"])
    similar_user_N_df["id"] = similar_user_N_df["id"].map(user_id_map)
    similar_user_N_df = similar_user_N_df.merge(df_user, on="id")

    print(f"User {user_id}: Top {similar_userN} similar students ...")
    print()
    for index, row in similar_user_N_df.reset_index().iterrows():
        print(f"{index+1}. Student ID: {row['id']} | Name: {row['fullName']} | Faculty: {row['faculty']} | Confidence: {row['confidence']:.4f}")
    

# Recommend

In [18]:
USER_ID = "20204876"

In [19]:
get_basic_info(USER_ID)

Student ID: 20204876. Number of Engaged Events: 195

-- Recent Engaged Activities: 
1. TỌA ĐÀM: CẦU NỐI DOANH NGHIỆP CỰU SINH VIÊN VÀ SINH VIÊN BÁCH KHOA
BTC: Ban CTSV
Thể loại: Sinh hoạt chuyên đề
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9075/chi-tiet
2. BUỔI NÓI CHUYỆN CỦA HỌC GIẢ QUỐC TẾ ĐẠT GIẢI NOBEL
BTC: Ban CTSV
Thể loại: Sinh hoạt chuyên đề
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9064/chi-tiet
3. [SHCD] Tọa đàm với doanh nghiệp và hướng nghiệp dành cho sinh viên năm cuối
BTC: Ban CTSV
Thể loại: Hội thảo hướng nghiệp
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9033/chi-tiet
4. Tập huấn kiến thức, kỹ năng PCCC 12.2023
BTC: Ban CTSV
Thể loại: Sinh hoạt công dân
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9025/chi-tiet
5. Career Day 2023 - Ngày hội hướng nghiệp và việc làm tại Đại học Bách khoa Hà Nội
BTC: Ban CTSV
Thể loại: Hội thảo hướng nghiệp
Link: https://ctsv.hust.edu.vn/#/hoat-dong/8982/chi-tiet
6. Bài giảng đại chúng của UNESCO: Đạo đức trong Trí tuệ nhân tạo
BTC: Trường

In [20]:
get_recommend_activities(USER_ID)

User 20204876: Top 5 recommender ...
1.Title: Bài giảng đại chúng: Đổi mới với AI – Quá trình chuyển đổi tại các quốc gia đang phát triển và khám phá bản thân dựa trên khoa học máy tính
BTC: Trường Công nghệ Thông tin và Truyền thông
Type: Hội thảo khoa học
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9068/chi-tiet
Confidence: 0.7990

2.Title: Tham gia Lễ khai mạc chương trình Hiến máu tình nguyện "CHỦ NHẬT ĐỎ lần thứ XVI - BÁCH KHOA NGHÌN GIỌT HI VỌNG lần thứ XXX"
BTC: Đoàn TNCS Hồ Chí Minh ĐH BKHN
Type: Hiến máu nhân đạo
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9074/chi-tiet
Confidence: 0.7330

3.Title: Tham gia các hoạt động, check in tại cây thông và Booth Chụp ảnh NOEL 2023: “APRICITY"
BTC: Đoàn TNCS Hồ Chí Minh ĐH BKHN
Type: Hoạt động văn nghệ
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9071/chi-tiet
Confidence: 0.5159

4.Title: HỘI THI TIẾNG HÁT CÔNG ĐOÀN NĂM 2023
BTC: Đại học Bách Khoa Hà Nội
Type: Hoạt động văn nghệ
Link: https://ctsv.hust.edu.vn/#/hoat-dong/9080/chi-tiet
Confi

In [21]:
get_similar_users(USER_ID)

User 20204876: Top 10 similar students ...

1. Student ID: 20200436 | Name: Phạm Quang Nghĩa | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9850
2. Student ID: 20204849 | Name: Đinh Ngọc Quân | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9723
3. Student ID: 20204787 | Name: Hà Hiểu Thành | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9718
4. Student ID: 20200585 | Name: Bùi Văn Thành | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9658
5. Student ID: 20200349 | Name: Nguyễn Thị Linh | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9651
6. Student ID: 20204992 | Name: Nguyễn Duy Khánh | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9643
7. Student ID: 20204572 | Name: Nguyễn Đăng Khoa | Faculty: Trường Công nghệ Thông tin và Truyền thông | Confidence: 0.9596
8. Student ID: 20200508 | Name: Nguyễn Minh Quân | Faculty: Trường Công nghệ Thông tin và Truyền 